In [55]:
import snscrape.modules.twitter as sntwitter
import pandas as pd


In [56]:
hashtag = "#SupremeCourt"
num_tweets = 10


In [57]:
tweets_list = []


In [58]:
for i, tweet in enumerate(sntwitter.TwitterSearchScraper(hashtag + ' lang:en').get_items()):
    if i >= num_tweets:
        break
    tweets_list.append([tweet.id, tweet.conversationId, tweet.date, tweet.user.id, tweet.user.username, 
                        tweet.user.displayname, tweet.place, tweet.rawContent, tweet.lang, tweet.mentionedUsers,
                       tweet.links, tweet.media, tweet.replyCount, tweet.retweetCount, 
                       tweet.likeCount, tweet.hashtags, tweet.cashtags, tweet.source, tweet.retweetedTweet, 
                       tweet.quotedTweet, tweet.inReplyToUser,tweet.inReplyToTweetId, tweet.viewCount])


In [59]:
tweets_df = pd.DataFrame(tweets_list, columns=['id', 'conversation_id', 'date','user_id', 'username', 
                                               'name', 'place', 'tweet', 'language', 'mentions',
                                               'urls', 'photos', 'replies_count', 'retweets_count', 
                                               'likes_count','hashtags', 'cashtags', 'source', 'retweet', 
                                               'quote_url','reply_to', 'reply_to_id','view_count'])


In [60]:
tweets_df.to_csv('custom_twitter_trend_dataset.csv', index=False)


In [61]:
df = pd.read_csv('./tweets_24_notendencias_raw.csv', nrows=1)
print(df.columns)


Index(['Unnamed: 0.1', 'Unnamed: 0', 'id', 'conversation_id', 'created_at',
       'date', 'time', 'timezone', 'user_id', 'username', 'name', 'place',
       'tweet', 'language', 'mentions', 'urls', 'photos', 'replies_count',
       'retweets_count', 'likes_count', 'hashtags', 'cashtags', 'link',
       'retweet', 'quote_url', 'video', 'thumbnail', 'near', 'geo', 'source',
       'user_rt_id', 'user_rt', 'retweet_id', 'reply_to', 'retweet_date',
       'translate', 'trans_src', 'trans_dest'],
      dtype='object')


In [62]:
df = pd.read_csv('./custom_twitter_trend_dataset.csv')
print(df.columns)


Index(['id', 'conversation_id', 'date', 'user_id', 'username', 'name', 'place',
       'tweet', 'language', 'mentions', 'urls', 'photos', 'replies_count',
       'retweets_count', 'likes_count', 'hashtags', 'cashtags', 'source',
       'retweet', 'quote_url', 'reply_to', 'reply_to_id', 'view_count'],
      dtype='object')


In [63]:
df = pd.read_csv('./tweets_24_notendencias_raw.csv', nrows=1)
df.head()


,Unnamed: 0.1,Unnamed: 0,id,conversation_id,created_at,date,time,timezone,user_id,username,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,0,105780,1232077745083027456,1231738733159964672,2020-02-24 23:59:59 Hora estándar romance,2020-02-24,23:59:59,100.0,156437000.0,germanlorenz21,...,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'diegojokas', 'name': 'diego ...",NaN,NaN,NaN,NaN


In [64]:
df = pd.read_csv('./custom_twitter_trend_dataset.csv')
df.head()


,id,conversation_id,date,user_id,username,name,place,tweet,language,mentions,...,retweets_count,likes_count,hashtags,cashtags,source,retweet,quote_url,reply_to,reply_to_id,view_count
0,1640997303509999618,1640997303509999618,2023-03-29 08:40:20+00:00,1152376540631576577,OfficialSafi,Sami safi 🥶,NaN,Elections are must for a functional democracy....,en,NaN,...,0,0,['SupremeCourt'],NaN,"<a href=""http://twitter.com/download/iphone"" r...",NaN,NaN,NaN,NaN,NaN
1,1640997297684135936,1640997297684135936,2023-03-29 08:40:18+00:00,4558887513,Numrahkhalid,NK,NaN,"Please #SupremeCourt, take a stand! You can no...",en,NaN,...,0,0,"['SupremeCourt', 'NationSupportsCJP', 'BehindY...",NaN,"<a href=""http://twitter.com/download/iphone"" r...",NaN,https://twitter.com/Jhagra/status/164064548103...,NaN,NaN,NaN
2,1640997294257639430,1640997294257639430,2023-03-29 08:40:17+00:00,828841083257778176,latestly,LatestLY,NaN,Supreme Court Issues Notice on Delhi Governmen...,en,NaN,...,0,0,"['SupremeCourt', 'Aldermen', 'MCD', 'DelhiLG',...",NaN,"<a href=""https://about.twitter.com/products/tw...",NaN,NaN,NaN,NaN,NaN
3,1640997252788256768,1640997252788256768,2023-03-29 08:40:07+00:00,138347068,TheSiasatDaily,The Siasat Daily,NaN,The amendments which have been proposed in the...,en,NaN,...,0,0,"['SupremeCourt', 'Pakistan']",NaN,"<a href=""https://app.socialpilot.co/"" rel=""nof...",NaN,NaN,NaN,NaN,NaN
4,1640997240041885698,1640997240041885698,2023-03-29 08:40:04+00:00,4558887513,Numrahkhalid,NK,NaN,"Please #SupremeCourt, take a stand! You can no...",en,NaN,...,0,0,"['SupremeCourt', 'NationSupportsCJP', 'BehindY...",NaN,"<a href=""http://twitter.com/download/iphone"" r...",NaN,https://twitter.com/PTIofficial/status/1640933...,NaN,NaN,NaN


In [67]:
json_object = df.to_json()

print(json_object)


{"id":{"0":1640997303509999618,"1":1640997297684135936,"2":1640997294257639430,"3":1640997252788256768,"4":1640997240041885698,"5":1640997232639090688,"6":1640997211520598016,"7":1640997198363230208,"8":1640997168751300610,"9":1640997143270809600},"conversation_id":{"0":1640997303509999618,"1":1640997297684135936,"2":1640997294257639430,"3":1640997252788256768,"4":1640997240041885698,"5":1640997232639090688,"6":1640997211520598016,"7":1640997198363230208,"8":1640997168751300610,"9":1640997143270809600},"date":{"0":"2023-03-29 08:40:20+00:00","1":"2023-03-29 08:40:18+00:00","2":"2023-03-29 08:40:17+00:00","3":"2023-03-29 08:40:07+00:00","4":"2023-03-29 08:40:04+00:00","5":"2023-03-29 08:40:03+00:00","6":"2023-03-29 08:39:58+00:00","7":"2023-03-29 08:39:55+00:00","8":"2023-03-29 08:39:47+00:00","9":"2023-03-29 08:39:41+00:00"},"user_id":{"0":1152376540631576577,"1":4558887513,"2":828841083257778176,"3":138347068,"4":4558887513,"5":85617284,"6":1228093296,"7":1231847789447045120,"8":45588

In [71]:
import json
from pathlib import Path

# jsonobjc = json.dumps(json_object)
# data_folder = Path("")
# file_to_open = data_folder / "jsonobjtrend.json"
# file = open(file_to_open, "w")
# file.write(jsonobjc)
# file.close()

if (json_object):
    # Writing to json
    data_folder = Path("")
    file_to_open = data_folder / "jsonobjtrend.json"
    file = open(file_to_open, "w")
    file.write(json_object)
    file.close()
